In [1]:
import json
import numpy as np
import torch
import torchvision
import shap


2024-01-15 13:38:55.685672: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-15 13:38:55.813779: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-01-15 13:38:55.813794: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-01-15 13:38:56.521543: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = torchvision.models.mobilenet_v2(pretrained=True, progress=False)
model.to(device)
model.eval()


In [6]:
X, y = shap.datasets.imagenet50() # y nie będzie potrzebne, zawiera odwołania do obrazów, które tu nie są wykorzystywane
url = "https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json"
with open(shap.datasets.cache(url)) as file:
    class_names = [v[1] for v in json.load(file).values()]
print(len(X),len(class_names)) # sprawdzamy czy się udało: 50, 1000


50 1000


Funkcja rotująca kształt tensora, w zależności od tego czy mamy kolekcję obrazów (4 wymiary, n,color,height,width) czy trzy (color,height,width). Funkcja pozostawia n bez zmian (jeśli jest) i przekształca klasyczny obraz (kanał, wiersz, kolumna)  poprzez przeniesienie c na koniec, tak jak w definicji tensora
```
  n number
| c color
| h height rows
V w width cols

n
h
w
c
```

In [7]:
#funkcja sprawdza czy mamy 4 wymiary (wiele obrazów) czy trzy wimiary (jeden obraz)

def nhwc_to_nchw(x: torch.Tensor) -> torch.Tensor:
    if x.dim() == 4:
        x = x if x.shape[1] == 3 else x.permute(0, 3, 1, 2)
    elif x.dim() == 3:
        x = x if x.shape[0] == 3 else x.permute(2, 0, 1)
    return x


def nchw_to_nhwc(x: torch.Tensor) -> torch.Tensor:
    if x.dim() == 4:
        x = x if x.shape[3] == 3 else x.permute(0, 2, 3, 1)
    elif x.dim() == 3:
        x = x if x.shape[2] == 3 else x.permute(1, 2, 0)
    return x

Natępnie ręcznie obrazy są skalowane do przedziału (0,1) a następnie standaryzowane, względem wartości średnich dla kanałów dla pretrenowanej sieci. Proces ten jest połączony w potok (pipe) metodą `Compose`

In [8]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# Rotacja jest potrzebna aby zastosować transformację obrazu w sposób zwektoryzowany
transform = [
    torchvision.transforms.Lambda(nhwc_to_nchw),
    torchvision.transforms.Lambda(lambda x: x * (1 / 255)),
    torchvision.transforms.Normalize(mean=mean, std=std),
    torchvision.transforms.Lambda(nchw_to_nhwc),
]

# transformacja odwrotna służy do wykonania destandaryzacji
inv_transform = [
    torchvision.transforms.Lambda(nhwc_to_nchw),
    torchvision.transforms.Normalize(
        mean=(-1 * np.array(mean) / np.array(std)).tolist(),
        std=(1 / np.array(std)).tolist(),
    ),
    torchvision.transforms.Lambda(nchw_to_nhwc),
]

#połączenie w pipe
transform = torchvision.transforms.Compose(transform)
inv_transform = torchvision.transforms.Compose(inv_transform)

Funkcja `predict`, zwraca zawiera obraz (np.array) na tensor, zgodny z urządzeniem (device na którym wykonywana jest predykcja)

In [9]:
def predict(img: np.ndarray) -> torch.Tensor:
    img = nhwc_to_nchw(torch.Tensor(img))
    img = img.to(device)
    output = model(img)
    return output

In [10]:
Xtr = transform(torch.Tensor(X))
out = predict(Xtr[7:8])
classes = torch.argmax(out, axis=1).cpu().numpy() # używam cpu bo mi CUDA nie działa na Waylandzie
print(f"Classes: {classes}: {np.array(class_names)[classes]}")

Classes: [643]: ['mask']


Budowanie właściwego explainera oraz maskera. Używamy obrazu rozmytego kernelem 64,64 na obrazie o ksztłcie kolekcji obrazów (obliczenia trochę trwają)

In [17]:
masker_blur = shap.maskers.Image("blur(64,64)", Xtr[0].shape)
# funkcja predict, masker output, lista nazw klas
explainer = shap.Explainer(predict, masker_blur, output_names=class_names)